In [ ]:
# test script for complete get_nse_options

import pandas as pd
from ib_insync import *
util.startLoop()
from helper import get_connected, get_nse_options, get_nses

ib = get_connected('nse', 'live')

qlm = get_nses(ib)
undContracts = qlm[0]
lots_dict = qlm[1]
margins_dict = qlm[2]

tickers = ib.reqTickers(*undContracts)
undPrices = {t.contract.symbol: t.marketPrice() for t in tickers} # {symbol: undPrice}

u = undContracts[0]
p = list(undPrices.values())[0]
z = list(lots_dict.values())[0]
m = list(margins_dict.values())[0]

get_nse_options(ib, u, p, z, m)

ib.disconnect()

import pandas as pd
pd.read_pickle('../data/nse/ACC.pkl').head() 

In [ ]:
# test script for one symbol get_nse_option

from ib_insync import *
util.startLoop()
from helper import get_connected, get_nses
ib = get_connected('nse', 'live')

symbol = 'PNB'
lotsize = 7000
c = ib.qualifyContracts(Stock(symbol, 'NSE'))[0]
ib.reqSecDefOptParams(underlyingSymbol=symbol, futFopExchange='', underlyingSecType='STK', underlyingConId=c.conId)

strikes = (57.5, 60.0, 62.5)
os = [Option('PNB', '20190627', s, 'P', 'NSE') for s in strikes]
ocs = ib.qualifyContracts(*os)
order = Order(action='SELL', orderType='MKT', totalQuantity=lotsize, whatIf=True)

from helper import get_nses
qlm = get_nses(ib)

qlm[2]['PNB']

from itertools import repeat
{c.conId: ib.whatIfOrder(c, Order(action='SELL', orderType='MKT', \
                                 totalQuantity=abs(ls), whatIf=True)).initMarginChange
 for c, ls in (zip(ocs, repeat(lotsize)))}

ib.disconnect()

In [ ]:
# get_snp_remqty.py program test script
def get_snp_remqty(ib):
    '''generates the remaining quantities dictionary
    Args:
        (ib) as connection object
    Returns:
        remqty as a dictionary of {symbol: remqty}
        '''
    exchange = 'SMART'
    snp_assignment_limit = 50000
    lotsize=100
    
    # get the list of underlying contracts
    undContracts = get_snps(ib)
    c_dict = {u.symbol: u for u in undContracts} # {symbol: contract}
    
    tickers = ib.reqTickers(*undContracts)
    undPrices = {t.contract.symbol: t.marketPrice() for t in tickers} # {symbol: undPrice}

    df_und = \
        pd.DataFrame.from_dict(undPrices, orient='index', columns=['undPrice']).\
        join(pd.DataFrame.from_dict(c_dict, orient='index', columns=['undContract'])).dropna()
    
    df_und = df_und.assign(lotsize=lotsize)
    
    # remaining quantities in entire snp per assignment limit
    df_und = df_und.assign(remq=(snp_assignment_limit/(df_und.lotsize*df_und.undPrice)).astype('int')) 
    
    # from portfolio
    #_______________
    
    p = util.df(ib.portfolio()) # portfolio table
    
    # extract option contract info from portfolio table
    dfp = pd.concat([p, util.df([c for c in p.contract])[util.df([c for c in p.contract]).columns[:7]]], axis=1).iloc[:, 1:]
    dfp = dfp.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

    # get the underlying's margins, lots, prices and positions
    pos_dict = dfp.groupby('symbol')['position'].sum().to_dict()
    p_syms = {s for s in dfp.symbol}
    p_undc = {s: ib.qualifyContracts(Stock(s, exchange, currency='USD')) for s in p_syms}   # {symbol: contracts} dictionary
    undmlist = [get_margins(ib, u, 100) for u in p_undc.values()]                 # {contract: margin} dictionary
    p_undMargins = {k.symbol: v for i in undmlist for k, v in i.items()}        # {symbol: margin} dictionary
    p_undPrices = {u: undPrices[u] for u in p_syms}    #{symbol: undPrice} dictionary
    
    dfp1 = pd.DataFrame.from_dict(p_undc, orient='index', columns=['contract']). \
        join(pd.DataFrame.from_dict(p_undMargins, orient='index', columns=['undmargin'])). \
        join(pd.DataFrame.from_dict(p_undPrices, orient='index', columns=['undPrice'])). \
        join(pd.DataFrame.from_dict(pos_dict, orient='index', columns=['position']))
    
    dfp1 = dfp1.assign(undLot=1)
    
    dfp1 = dfp1.assign(qty=(dfp1.position/dfp1.undLot).astype('int'))
    
    # make the blacklist
    #___________________
    remqty_dict = pd.DataFrame(df_und.loc[dfp1.index].remq + dfp1.qty).to_dict()[0] # remq from portfolio
    remqty_dict = {k:(v if v > 0 else 0) for k, v in remqty_dict.items()} # portfolio's remq with negative values removed
    blacklist = [k for k, v in remqty_dict.items() if v <=0] # the blacklist
    df_und.remq.update(pd.Series(remqty_dict)) # replace all underlying with remq of portfolio
    remqty = df_und.remq.to_dict() # dictionary
    return remqty

In [ ]:
import pandas as pd
from ib_insync import *
util.startLoop()
from helper import get_connected, get_snps, get_margins
# ib = get_connected('snp', 'live')
remqty = get_snp_remqty(ib)
ib.disconnect()

In [ ]:
import pandas as pd
import numpy as np

#***      Function to print python code in Jupyter   ****
#________________________________________________________

def display_py(code):
    """Displays python file code in Jupyter
    Arg: (string from py file) code
    Output: code formatted for jupyter
    Usage:
    with open(myfile) as f:
         code = f.read()
    display_py(code)
    """
    from pygments import highlight
    from pygments.lexers import PythonLexer
    from pygments.formatters import HtmlFormatter
    import IPython
    
    formatter = HtmlFormatter()

    html_code = highlight(code, PythonLexer(), HtmlFormatter())
    styled_html = '<style type="text/css">{}</style>{}'.format(formatter.get_style_defs('.highlight'), html_code)
    ipython_code = IPython.display.HTML(styled_html)

    return ipython_code

In [ ]:
from helper import get_connected
??get_connected

In [38]:
# get the correct margin from median of strikes (test)

fspath = '../data/nse/'

ib = get_connected('nse', 'live') # close the connection and delete it final function

import pandas as pd
from itertools import product

from ib_insync import *
util.startLoop()

from helper import get_connected, catch

exchange = 'NSE'

tp = pd.read_html('https://www.tradeplusonline.com/Equity-Futures-Margin-Calculator.aspx')

df_tp = tp[1][2:].iloc[:, :-1]
df_tp = df_tp.iloc[:, [0,1,5]]
df_tp.columns=['nseSymbol', 'lot', 'margin']

cols = df_tp.columns.drop('nseSymbol')
df_tp[cols] = df_tp[cols].apply(pd.to_numeric, errors='coerce') # convert lot and margin to numeric

df_slm = df_tp.copy()

# Truncate to 9 characters for ibSymbol
df_slm['ibSymbol'] = df_slm.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MMFIN', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_slm.ibSymbol = df_slm.ibSymbol.replace(ntoi)

# separate indexes and equities, eliminate discards from df_slm
indexes = ['NIFTY50', 'BANKNIFTY']
discards = ['NIFTYMID5', 'NIFTYIT', 'LUPIN']
equities = sorted([s for s in df_slm.ibSymbol if s not in indexes+discards])

symbols = equities+indexes

cs = [Stock(s, exchange) if s in equities else Index(s, exchange) for s in symbols]

qcs = ib.qualifyContracts(*cs) # qualified underlyings

qcs_dict = {q.symbol: q for q in qcs}

lots_dict = [v for k, v in df_slm[['ibSymbol', 'lot']].set_index('ibSymbol').to_dict().items()][0]

m_dict = [v for k, v in df_slm[['ibSymbol', 'margin']].set_index('ibSymbol').to_dict().items()][0] # from website

# get the underlying prices
tickers = ib.reqTickers(*qcs)
undPrices = {t.contract.symbol: t.marketPrice() for t in tickers} # {symbol: undPrice}

# get the option chains
ch_list = [(q.symbol, 'IND', q.conId) 
           if q.symbol in indexes 
           else (q.symbol, 'STK', q.conId) 
           for q in qcs]

chains = {s: ib.reqSecDefOptParams(underlyingSymbol=s, underlyingSecType=t, underlyingConId=c, futFopExchange='') for s, t, c in ch_list}

# generate whatif contract-orders for margins
co = [(k, min(v[0].expirations), 
  min(v[0].strikes, key=lambda 
      x:abs(x-undPrices[k])), 'P', 'NSE', lots_dict[k]) 
  for k, v in chains.items()]

m_dict_ib = {c[0]:catch(lambda: ib.whatIfOrder(Option(c[0], c[1], c[2], c[3], c[4]), 
                     Order(action='SELL', orderType='MKT', totalQuantity=c[5], whatIf=True)).initMarginChange)
 for c in co}

margins = {k: max(v, float(m_dict_ib[k])) for k, v in m_dict.items() if k not in discards}

Peer closed connection
clientId 1 already in use?
API connection failed: TimeoutError()


Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests


In [50]:
# contracts, lots, margins, undPrices dataframe
df_clmu = pd.concat([pd.DataFrame.from_dict(qcs_dict, orient='index'), 
           pd.DataFrame.from_dict(lots_dict, orient='index'), 
           pd.DataFrame.from_dict(margins, orient='index'),
           pd.DataFrame.from_dict(undPrices, orient='index')], axis=1, sort=False)

df_clmu.columns=['contract', 'lot', 'margin', 'undPrice']

df_clmu.to_pickle(fspath+'_lot_margin.pickle')

In [ ]:
ib.disconnect()

In [2]:
%%time
from ib_insync import *
util.startLoop()

from helper import get_connected, get_nses
with get_connected('nse', 'live') as ib:
    nses = get_nses(ib)

Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests


Wall time: 4min 38s


In [37]:
pd.concat([pd.DataFrame.from_dict(nses[0], orient='index'), 
           pd.DataFrame.from_dict(nses[1], orient='index'), 
           pd.DataFrame.from_dict(nses[2], orient='index'),
           pd.DataFrame.from_dict(undPrices, orient='index')], axis=1, sort=False)

NameError: name 'undPrices' is not defined

In [27]:
help(pd.DataFrame.from_dict)

Help on method from_dict in module pandas.core.frame:

from_dict(data, orient='columns', dtype=None, columns=None) method of builtins.type instance
    Construct DataFrame from dict of array-like or dicts.
    
    Creates DataFrame object from dictionary by columns or by index
    allowing dtype specification.
    
    Parameters
    ----------
    data : dict
        Of the form {field : array-like} or {field : dict}.
    orient : {'columns', 'index'}, default 'columns'
        The "orientation" of the data. If the keys of the passed dict
        should be the columns of the resulting DataFrame, pass 'columns'
        (default). Otherwise if the keys should be rows, pass 'index'.
    dtype : dtype, default None
        Data type to force, otherwise infer.
    columns : list, default None
        Column labels to use when ``orient='index'``. Raises a ValueError
        if used with ``orient='columns'``.
    
        .. versionadded:: 0.23.0
    
    Returns
    -------
    pandas.Data